In [46]:
import cv2
import os
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import matplotlib
from BoundingBoxes import BoundingBoxes
from BoundingBox import BoundingBox
from utils import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

pkl_file = "/home/yuval/Documents/XNOR/sealnet/metrics/results.pkl"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
def print_bbs(bbs):
    ct = len(bbs._boundingBoxes)
    class_ct = {}
    for bb in bbs._boundingBoxes:
        if bb.getClassId() not in class_ct:
            class_ct[bb.getClassId()] = 0

        class_ct[bb.getClassId()] += 1
    for c in class_ct:
        print("%s %d" % (c, class_ct[c]))
    print("%d total" % ct)
    print("")

def print_models(models):
    print("Total detections:")
    for model in models:
        print(model)
        bbs = models[model]
        print_bbs(bbs)

In [32]:
df = pd.read_csv("/home/yuval/Documents/XNOR/sealnet/models/darknet/full_detections.csv", delimiter='\t')
unique_ims = set(df["file"].values[0:])
models = {}
if os.path.exists(pkl_file):
    models = pickle.load(open(pkl_file, "rb"))
else:
    for i, det in df.iloc[1:].iterrows():
        imageName=det["file"]
        model=det["weights"]
        label=det["label"]
        confidence=det["confidence"]
        x=det["x"]
        y=det["y"]
        w=det["w"]
        h=det["h"]
        if not model in models:
            models[model] = BoundingBoxes()

        box = BoundingBox(imageName, label,x,y,w,h,
                          classConfidence=confidence, bbType=BBType.Detected, model=model)
        models[model].addBoundingBox(box)
    pickle.dump( models, open( pkl_file, "wb" ) )
all_together =BoundingBoxes()
for model in models:
    for box in models[model]._boundingBoxes:
        all_together.addBoundingBox(box)    

In [ ]:
from Evaluator import Evaluator
bbs_conf = None
evaluator = Evaluator()
for i, im_name in enumerate(unique_ims):
    if i !=2:
        continue
    bblist = all_together.getBoundingBoxesByImageName(im_name)
    
    for i in range(len(bblist)):
        ious = evaluator._getAllIOUs(bblist[i], bblist[:i] + bblist[i+1:])
        for iou in ious:
            print(iou)
            
    bbs = BoundingBoxes()
    bbs.addBoundingBoxes(bblist)
    bbs_conf = bbs.filter_confidence(.1)
bbs_conf._boundingBoxes

In [41]:
models = {}
for bb in bblist
    if not bb_model in models:
        models[bb_model = ]

[1, 2, 4]

In [83]:
def iou(box, clusters):
    x = np.minimum(clusters[:, 0], box[0])
    y = np.minimum(clusters[:, 1], box[1])

    intersection = x * y
    box_area = box[0] * box[1]
    cluster_area = clusters[:, 0] * clusters[:, 1]

    iou_ = intersection / (box_area + cluster_area - intersection)
    
    return iou_

def kmeans(boxes, k, dist=np.median):
    boxes2= np.array(boxes)
    boxes = np.zeros((len(boxes2), 4))
    for i, b in enumerate(boxes2):
        bb=np.array(b.getAbsoluteBoundingBox(format=BBFormat.XYX2Y2), dtype=np.int)
        boxes[i] = bb

    rows = len(boxes)

    distances = np.empty((rows, k))
    last_clusters = np.zeros((rows,))

    np.random.seed()

    clusters = boxes[np.random.choice(rows, k, replace=False)]

    while True:
        for row in range(rows):
            distances[row] = 1 - iou(boxes[row], clusters)

        nearest_clusters = np.argmin(distances, axis=1)

        if (last_clusters == nearest_clusters).all():
            break

        for cluster in range(k):
            clusters[cluster] = dist(boxes[nearest_clusters == cluster], axis=0)

        last_clusters = nearest_clusters

    return clusters
x=kmeans(bblist, 4)
print(x)

[[5687.  215. 5750.  313.]
 [5692.  215. 5749.  309.]
 [5678. 2385. 5735. 2418.]
 [4843. 3959. 4895. 4005.]
 [5685.  215. 5750.  300.]]


In [34]:
im = None
bbs = Nones
for im_name in unique_ims:
    imbbs = all_together.getBoundingBoxesByImageName(im_name)
    im = cv2.imread(os.path.join("/data/raw_data/TrainingAnimals_ColorImages/", im_name))
    for bb in imbbs:
        if bb._classConfidence > .15 or bb.getClassId() == "Polar Bear" :
            box_label = "%s %.4f" % (bb.getClassId(), bb._classConfidence)
            add_bb_into_image(im, bb, label=box_label)
    
    im = im
    bbs = imbbs
    break
plt.figure(figsize = (20,20))
plt.imshow(im,cmap='gray', interpolation='nearest', aspect='auto')
plt.show()


NameError: name 'Nones' is not defined